# Post-hoc model analysis

In [12]:
import sys
import os
import torch
import pandas as pd

from confection import Config
from pathlib import Path
from torch.utils.data import DataLoader

# update the path so we can directly import code from the DVlog
sys.path.append(os.path.dirname(os.path.abspath(os.path.join(os.getcwd(), os.pardir, "DVlog"))))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, "DVlog")))

from DVlog.evaluate import evaluate_model
from DVlog.models.model import UnimodalDVlogModel
from DVlog.utils.dataloaders import MultimodalEmbeddingsDataset
from DVlog.utils.metrics import calculate_performance_measures, calculate_gender_performance_measures
from DVlog.utils.util import ConfigDict, validate_config, process_config, set_seed

## load in the annotations file

In [13]:
text_dir = Path(r"../data/dvlog_text")
annotations_file = Path(r"../DVlog/dataset/dvlog_labels_v2.csv")

In [14]:
# load in the annotation labels
df_annotations = pd.read_csv(annotations_file)

# select the test set
df_annotations = df_annotations[df_annotations["dataset"] == "test"]
df_annotations.head()

,video_id,label,gender,dataset
1,1,1,f,test
4,4,1,f,test
6,6,1,f,test
12,13,1,f,test
13,15,1,f,test


## Evaluate detection models
Load in both the normal and keyword filtered model and evaluate them using the test set in order to retrieve the prediction information

In [16]:
models_path = Path(r"../DVlog/trained_models")
normal_model_config = Path(r"../DVlog/model_configs/unimodal/unimodal_mpnet_sent_normal.cfg")
keyw_removed_model_config = Path(r"../DVlog/model_configs/unimodal/unimodal_mpnet_sent_keyw.cfg")

In [18]:
# load in the first config dict and evaluate the normal model
config = Config().from_disk(normal_model_config)
config_dict = process_config(config)

# overwrite the annotations_file + data_dir
config_dict.annotations_file = annotations_file
config_dict.data_dir = Path("../DVlog/dataset/sent-embeddings-dataset")

# setup the model paths
model_dir_path = Path(os.path.join(models_path, config_dict.model_name))
saved_model_path = Path(os.path.join(model_dir_path, f"model_{config_dict.model_name}.pth"))

# set the seed
set_seed(42)

# setup the original model
saved_model = UnimodalDVlogModel((config_dict.sequence_length, config_dict.encoder1_dim),
                                   d_model=config_dict.dim_model, n_heads=config_dict.uni_n_heads, use_std=config_dict.detectlayer_use_std)

# load in the parameters and set the model to evaluation mode
saved_model.load_state_dict(torch.load(saved_model_path))
saved_model.eval()

# setup the dataset + loader
test_data = MultimodalEmbeddingsDataset("test", config_dict, to_tensor=True, with_protected=True)
test_dataloader = DataLoader(test_data, batch_size=config_dict.batch_size, shuffle=True)

# evaluate both models and retrieve the raw prediction information
norm_pred, norm_truth, norm_protected, norm_video_ids = evaluate_model(saved_model, test_dataloader, config_dict,
                                                                       unpriv_feature="m", verbose=False, get_raw_preds=True)

# retrieve the evaluation metrics to check the model
_, w_precision, w_recall, w_fscore, _ = calculate_performance_measures(y_labels, predictions)
gender_metrics = calculate_gender_performance_measures(y_labels, predictions, protected)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset\\sent-embeddings-dataset\\250\\sent_mpnet_normal.npy'

In [ ]:
keyw_pred, keyw_truth, keyw_protected, keyw_video_ids = evaluate(normal_model_config, trained_model_dir, 0,
                                                                 unpriv_feature, verbose=False, get_raw_predictions=True)